## Give me the best deck with Champion X

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'./../data/raw/lor_ranked_data_all.csv') # read in the data

In [3]:
champion = 'Aurelion Sol'
date = '2022-06-16'
n = 50

In [4]:
def best_deck_by_champ(df, champion, date, n):
    df_date = df[df['game_start_time_utc'] > date]
    champ_list = df_date[df_date['archetype'].str.contains(champion)]
    archetypes = champ_list['deck_code'].value_counts() # find the unique archetypes being played
    archetypes = archetypes[archetypes > n] #only keep popular deck_codes
    popular_list = list(archetypes.index) # make a list of popular deck_codes
    champ_list = champ_list.loc[df['deck_code'].isin(popular_list)] # filter df to only keep popular archetypes
    df_champ = champ_list.groupby(['deck_code', 'game_outcome']).size()
    df_champ = df_champ.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
    df_champ.drop('loss', level=1, axis=0, inplace=True) #drop loss %
    df_champ = df_champ.droplevel(level=1) #drop win/loss
    df_champ = df_champ.sort_values(ascending=False)
    
    df_small = df_date[['deck_code', 'archetype']]
    y = pd.DataFrame(df_champ)
    
    inner = pd.merge(y, df_small, on='deck_code', how='inner')#.value_counts()
    inner = inner.value_counts()
    df_champ = inner.sort_values(ascending=False)
    
    return df_champ

In [5]:
best_deck_by_champ(df, champion, date, n)

deck_code                                                                                 0     archetype                        
CICQCAIADIAQMABEAIBQSAYOAICAADQUAQBQAAQGBMHAEAIFAAGAEAYJK5SAGAICAAAQCAYJEYBAIAACBA        48.0  Shyvana Aurelion Sol Jarvan IV       850
CIBQCAIBBQBAGAIKCYBQGCKAKZLQGAIEAEGAEAYBAMJQIAIBCQTSUNAEAEAQCFYBAMAQIAQEAECQ4BADBECRKF2U  50.0  Tryndamere Aurelion Sol Lissandra    228
CICACBABBYBQCAIMCQ2AGAYBAMFBMBADBEEUAVSXAIAQGCKUAICACCQMAEAQGCJT                          47.0  Zoe Aurelion Sol (FR)                217
CIDACAIADIAQEAABAECQADACAQAA4FADAMAAMCYOAMBQSAYOK4BACBQAEQBAGCKVMQAQCAYJBU                45.0  Shyvana Aurelion Sol                 123
CIBQGAYBAMFBMBABAEBQYIJUAQBQSBKAKZLQEAIDBEEQEAIBCQVACAIDBFKA                              47.0  Zoe Aurelion Sol (FR)                116
CICACBABBYBACAIMGQBAGAIKCYBQGCIJIBLQIAIDAEBQCBIJBYBACAIUGIBQGCIXKRLAEAIDAEJQCBABBI        49.0  Zoe Aurelion Sol (FR)                 96
CICQCAIADIAQIAAUAECQADABAYACIBADAABAMCYOAMAQEAAB

In [6]:
# double check it worked
df_date = df[df['game_start_time_utc'] > date] #build dataframe you build in function
deck = y.index[0][0] # get deck code returned for deck #1 in the function
true_value = df_date.deck_code.str.count(deck).sum() #see how many times that deck actually appears in dataframe
output_value = y[0] # get how many times the function said that deck appears
true_value == output_value # compare values